In [11]:
import json
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sqlalchemy import URL
import numpy as np 
import plotly.graph_objects as go
from pathlib import Path
import os

# функция для прогона по всем ДФ

In [12]:
def calculete_stats_and_map(name):
    data = pd.read_csv(f"{os.getcwd()}\\Данные\\Исходные\\Переменные\\{name}", encoding = "windows-1251")
    data_for_date = data[["date_loc", "site_code", "site_name"]]

    max_data_for_date = data_for_date.groupby(["site_code", "site_name"]).max()
    max_data_for_date.rename(columns={"date_loc": 'date_loc_max'}, inplace=True)
    min_data_for_date = data_for_date.groupby(["site_code", "site_name"]).min()
    min_data_for_date.rename(columns={"date_loc": 'date_loc_min'}, inplace=True)
    
    date_stat = pd.concat([min_data_for_date, max_data_for_date], axis=1).reset_index()
    
    data_for_stats = data[["value", "site_code", 'site_name',]]

    data_for_loc = data[["site_code", "lon", "lat"]]
    data_for_loc = data_for_loc.groupby(["site_code"]).mean()
    # среднее
    mean_by_site = data_for_stats.groupby(["site_code", "site_name"]).mean()
    mean_by_site.rename(columns={"value": 'value_mean'}, inplace=True)
    # максимальное
    max_by_site = data_for_stats.groupby(["site_code", "site_name"]).max()
    max_by_site.rename(columns={"value": 'value_max'}, inplace=True)
    # минимальное
    min_by_site = data_for_stats.groupby(["site_code", "site_name"]).min()
    min_by_site.rename(columns={"value": 'value_min'}, inplace=True)
    # количество 
    amount_by_site = data_for_stats.groupby(["site_code", "site_name"]).count()
    amount_by_site.rename(columns={"value": 'value_amount'}, inplace=True)
    
    # Мерджим все дата фреймы для общей таблицы
    value_stat = pd.concat([mean_by_site, min_by_site, max_by_site, amount_by_site], axis=1).reset_index()
    stat = pd.concat([value_stat.set_index(["site_code", "site_name"]), date_stat.set_index(["site_code", "site_name"])], axis=1).reset_index()

    amount_by_site = amount_by_site.reset_index().set_index("site_code")
    for_mapping = pd.concat([amount_by_site, data_for_loc], axis=1).reset_index()

    for_mapping['name_map'] = for_mapping['site_name'].astype(str) + ' (' + for_mapping['site_code'].astype(str) + ') кол-во измерений:' + for_mapping['value_amount'].astype(str)

    for_mapping.to_csv(f"map_{name}", sep=";", encoding="cp1251")
    stat.to_csv(f"full_stat_{name}", sep=";", encoding="cp1251")
    

In [13]:

directory = f"{os.getcwd()}\\Данные\\Исходные\\Переменные\\"
# Создаем пустой список
files = []
# Добавляем файлы в список
files += os.listdir(directory)
# Выводим список файлов

In [14]:
for name in files:
    calculete_stats_and_map(name = name)